In [1]:
from pyuvdata import UVData, UVCal, utils
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import time
from glob import glob
import os
import ja

In [2]:
# Get the files which satisfy the sun cut
path = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458098/'
rawpath = '/lustre/aoc/projects/hera/H1C_IDR2/2458098/'
imagefits = glob(path+'zen.*.HH.calibrated.uvh5_image/*.image.image.fits')
uvh5stem = []
uvh5files = []
for i,imf in enumerate(imagefits):
    tmp = os.path.split(imf)[-1].split('.')
    uvh5stem.append(tmp[0]+'.'+tmp[1]+'.'+tmp[2])
    uvh5files.append(rawpath+uvh5stem[i]+'.HH.uvh5')
uvh5stem.sort()
uvh5files.sort()

In [3]:
Nfiles = len(uvh5files)

In [4]:
cal_path = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458098/'
out_path = '/lustre/aoc/projects/hera/jaguirre/PolarizedMosaics/Test/'

In [5]:
timer0 = ja.Timer()
timer= ja.Timer()
timer2 = ja.Timer()

In [6]:
#uvh5files = uvh5files[0:1]
#uvh5stem = uvh5stem[0:1]
#print(uvh5files)
#print(uvh5stem)

In [7]:
timer0.start()
for i,rf in enumerate(uvh5files):

    timer2.start()
    ja.progress_bar(i/Nfiles)
    
    raw_file = rf
    cal_file = cal_path+uvh5stem[i]+'.HH.smooth_abs.calfits'
    out_file_stem = out_path+uvh5stem[i]+'.HH.calibrated'
    print(raw_file)
    print(cal_file)
    print(out_file_stem)
    
    # Read the data
    timer.start()
    uvd = UVData()
    uvd.read(raw_file)
    timer.stop('Reading data took ')

    uvc = UVCal()
    uvc.read_calfits(cal_file)

    timer.start()
    uv_calibrated = utils.uvcalibrate(uvd, uvc, inplace=False)
    timer.stop('Applying calibration data took ')

    timer.start()
    uv_calibrated.write_uvh5(out_file_stem+'.uvh5', clobber = True)
    timer.stop('Writing uvh5')

    timer.start()
    uv_calibrated.write_uvfits(out_file_stem+'.uvfits',force_phase = True, spoof_nonessential = True)
    timer.stop('Writing uvfits')

    timer2.stop('One file took')
    
timer0.stop('Total time')

Progress: [####################] 98.2%
/lustre/aoc/projects/hera/H1C_IDR2/2458098/zen.2458098.64002.HH.uvh5
/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458098/zen.2458098.64002.HH.smooth_abs.calfits
/lustre/aoc/projects/hera/jaguirre/PolarizedMosaics/Test/zen.2458098.64002.HH.calibrated
Reading data took  53.455729722976685 sec
Applying calibration data took  22.44330668449402 sec
Writing uvh5 27.41802716255188 sec
The data are in drift mode and do not have a defined phase center. Phasing to zenith of the first timestamp.
Writing uvfits 47.374117612838745 sec
One file took 151.62191653251648 sec
Total time 8094.753732681274 sec
